# Final Project NLP Smart Scientific Content Auto-completion System


a text auto-complete system that supports scientists in writing articles in:
- completing some typed words/letters.
- Identifying words/phrases that are very unlikely to occur in some text contexts and suggesting proper replacements.


Karmel Salah
15/4/2022

# We will use Colab for a Free GPU so if you are using Colab frist connect to your drive to get the dataset


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#unzip the dataset folder
!unzip ./drive/MyDrive/dataset.zip 

Archive:  ./drive/MyDrive/dataset.zip
   creating: dataset/
  inflating: dataset/1012.2609.txt   
  inflating: dataset/1207.1847v1.txt  
  inflating: dataset/1209.3126v1.txt  
  inflating: dataset/1210.0852.txt   
  inflating: dataset/1305.6143.txt   
  inflating: dataset/1306.6802v2.txt  
  inflating: dataset/1308.0850v5.txt  
  inflating: dataset/1309.4168v1.txt  
  inflating: dataset/1312.3005v3.txt  
  inflating: dataset/1403.6652v2.txt  
  inflating: dataset/1404.3377v1.txt  
  inflating: dataset/1404.4326v1.txt  
  inflating: dataset/1405.4053v2.txt  
  inflating: dataset/1405.4273v1.txt  
  inflating: dataset/1406.1078v3.txt  
  inflating: dataset/1408.5882v2.txt  
  inflating: dataset/1409.0473v7.txt  
  inflating: dataset/1409.2329v5.txt  
  inflating: dataset/1409.3215v3.txt  
  inflating: dataset/1410.5329v4.txt  
  inflating: dataset/1410.8206v4.txt  
  inflating: dataset/1411.4555v2.txt  
  inflating: dataset/1412.1058v2.txt  
  inflating: dataset/1412.2007v2.txt  
  infla

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os, glob
import math
import matplotlib.pyplot as plt
import seaborn as sns
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package punkt to /home/karmel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/karmel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/karmel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/karmel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-05-09 17:02:53.047170: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such fil

In [2]:
# get the names of each folders
files_names = os.listdir('./dataset')
len(files_names)

1064

# Convert to DataFrame

In [3]:
df = pd.DataFrame([], columns = ["id", "articles"])
print(f"data frame is {df}")

data frame is Empty DataFrame
Columns: [id, articles]
Index: []


In [4]:
def convert_to_df(df, files_names):
        id = []
        articles= []
        # Define files place and read each sub file for cleaning
        for index, file in enumerate(files_names):
                # read each article files
                path = os.getcwd()
                file_path = '{}/dataset/{}'.format(path,file)
                with open(file_path,'r', encoding='utf-16',  errors="ignore") as f:
                    texts = f.read() 
                    articles.append(texts) 
                    id.append(index)
                    f.close()
        print("the ids", id)
        print("the articles", articles)
        df['id'] = id
        df["articles"] = articles
        return df


In [5]:
df = convert_to_df(df, files_names)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
df["articles"]

0       Statistical Machine Translation\n\nDraft of Ch...
1       Found in Translation:\n\nLearning Robust Joint...
2       Available online at www.sciencedirect.com\n\nS...
3       J OURNAL OF I NFORMATION S CIENCE AND E NGINEE...
4       The University of Sheﬃeld\n\nT. E. Dunning\n\n...
                              ...                        
1059    Multi-label Hate Speech and Abusive Language D...
1060    hULMonA (      ): The Universal Language Model...
1061    Mazajak: An Online Arabic Sentiment Analyser\n...
1062    2017 2nd International Conferences on Informat...
1063    INVESTIGATIVE RADIOLOGY Volume 28, Number 6, 4...
Name: articles, Length: 1064, dtype: object

In [7]:
limited_df = pd.DataFrame(df['articles'].iloc[0:50])

In [9]:
# count the number of words for each doc
limited_df['word_count'] = limited_df['articles'].apply(lambda x: len(str(x).split()))
limited_df["word_count"]

0     48052
1      7759
2      4223
3      8574
4     71427
5      8025
6      7159
7      2685
8     18542
9     13682
10     5449
11     3177
12     8716
13     7259
14     7244
15     6694
16     7691
17     8333
18     4001
19     9149
20    10087
21     5835
22     7457
23     6153
24     7287
25     8170
26     7348
27     6668
28     2837
29     3721
30     5282
31     7388
32     7437
33     8702
34     8665
35     3796
36     1819
37    17959
38    14282
39     6846
40     2847
41     6549
42     7226
43     7515
44     7194
45     6416
46     7524
47     3659
48     8019
49     7734
Name: word_count, dtype: int64

 # Text pre-processing:

In [10]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    # Remove leading and trailing spaces
    text= text.strip()  
    text= re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text= re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    # get ride of one letter word!
    text = re.sub(r'\b\w\b','',text) 
    
      
    return text

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/karmel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [13]:
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [15]:
import enchant

In [16]:
#Remove non English words
dictionary = enchant.Dict("en_US")
   
def remove_non_english(text):
        en_words= []
        for word in text:
            if dictionary.check(word):
                en_words.append(word)
        text = ' '.join(str(e) for e in en_words)
        return text

In [17]:
limited_df['english_text'] =limited_df['articles'].apply(lambda text: remove_non_english(text.split()))

In [18]:
limited_df['english_text'][0]

'Statistical Machine Translation Draft of Chapter Neural Machine Translation Center for Speech and Language Processing Department of Computer Science Johns Hopkins University 1st public draft August 2015 2nd public draft September 2017 2 Contents 13 Neural Machine Translation 5 13.1 A Short History . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5 13.2 Introduction to Neural Networks . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6 13.2.1 Linear Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7 13.2.2 Multiple Layers . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8 13.2.3 Non-Linearity . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9 13.2.4 Inference . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10 13.2.5 Back-Propagation Training . . . . . . . . . . . . . . . . . . . . . . . . . . . 11 13.2.6 . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [19]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [20]:
limited_df['clean_text'] = limited_df['english_text'].apply(lambda x: finalpreprocess(x))


,id,articles
0,0,Statistical Machine Translation\n\nDraft of Ch...
1,1,Found in Translation:\n\nLearning Robust Joint...
2,2,Available online at www.sciencedirect.com\n\nS...
3,3,J OURNAL OF I NFORMATION S CIENCE AND E NGINEE...
4,4,The University of Sheﬃeld\n\nT. E. Dunning\n\n...


In [22]:
articles = limited_df['clean_text']

In [24]:
raw_text = articles[0]

In [55]:
corpus = ''

for article in articles:
    corpus += article
    break

In [56]:
corpus{topic, : }

'statistical machine translation draft chapter neural machine translation center speech language processing department computer science john hopkins university st public draft august nd public draft september content neural machine translation short history introduction neural network linear model multiple layer non linearity inference back propagation training computation graph neural network computation graph gradient computation deep learn framework neural language model fee forward neural language model word embed inference training recurrent neural language model long short term memory model gate recurrent unit deep model neural translation model encoder decoder approach add alignment model train beam search content ensemble decode large vocabulary use monolingual data deep model guide alignment training model coverage adaptation add linguistic annotation multiple language pair alternate architecture neural network neural network attention self attention current challenge domain m

In [26]:
# creates mapping of unique characters to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

print(chars)
print("Char to int" , char_to_int)
print("Int to char", int_to_char)


[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Char to int {' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
Int to char {0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [27]:
# Prints the total characters and character vocab size
n_chars = len(raw_text)
n_vocab = len(chars)

print("The number of total characters are", n_chars)
print("\nThe character vocab size is", n_vocab)

The number of total characters are 138822

The character vocab size is 27


In [35]:
#Prepares dataset where the input is sequence of 100 characters and target is next character.
seq_length = 100

dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
          seq_in = raw_text[i:i + seq_length]
          seq_out = raw_text[i + seq_length]
          dataX.append([char_to_int[char] for char in seq_in])
          dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

 data X [19, 20, 1, 20, 9, 19, 20, 9, 3, 1, 12, 0, 13, 1, 3, 8, 9, 14, 5, 0, 20, 18, 1, 14, 19, 12, 1, 20, 9, 15, 14, 0, 4, 18, 1, 6, 20, 0, 3, 8, 1, 16, 20, 5, 18, 0, 14, 5, 21, 18, 1, 12, 0, 13, 1, 3, 8, 9, 14, 5, 0, 20, 18, 1, 14, 19, 12, 1, 20, 9, 15, 14, 0, 3, 5, 14, 20, 5, 18, 0, 19, 16, 5, 5, 3, 8, 0, 12, 1, 14, 7, 21, 1, 7, 5, 0, 16, 18, 15, 3]
 data Y 5
Total Patterns:  138722


In [38]:
from keras.utils import np_utils

# reshapes X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

print(X.shape)

# one hot encodes the output variable
y = np_utils.to_categorical(dataY)
print(y[0])

(138722, 100, 1)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]


In [39]:
embedding_dim =100
max_length =100

In [41]:
from keras.layers import Dropout

In [42]:
model = Sequential()
model.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          2700      
                                                                 
 lstm_1 (LSTM)               (None, 256)               365568    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 27)                6939      
                                                                 
Total params: 375,207
Trainable params: 375,207
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.fit(X, y, epochs = 20, batch_size=128)

Epoch 1/20


2022-05-09 17:29:21.629920: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 110977600 exceeds 10% of free system memory.


1084/1084 [==============================] - 451s 414ms/step - loss: 1.9136
Epoch 2/20
1084/1084 [==============================] - 435s 401ms/step - loss: 1.2530
Epoch 3/20
1084/1084 [==============================] - 456s 421ms/step - loss: 1.0920
Epoch 4/20
1084/1084 [==============================] - 673s 621ms/step - loss: 1.0119
Epoch 5/20
1084/1084 [==============================] - 719s 663ms/step - loss: 0.9586
Epoch 6/20
1084/1084 [==============================] - 722s 666ms/step - loss: 0.9177
Epoch 7/20
1084/1084 [==============================] - 705s 650ms/step - loss: 0.8832
Epoch 8/20
1084/1084 [==============================] - 722s 666ms/step - loss: 0.8544
Epoch 9/20
1084/1084 [==============================] - 764s 705ms/step - loss: 0.8280
Epoch 10/20
1084/1084 [==============================] - 738s 681ms/step - loss: 0.8044
Epoch 11/20
1084/1084 [==============================] - 724s 668ms/step - loss: 0.7805
Epoch 12/20
1084/1084 [=============================

In [49]:
# Generates the sequence similar to above methods. Gets the generated string using the model.
def predict_next_n_chars(pattern, n):
    for i in range(n):
      x = np.reshape(pattern, (1, len(pattern), 1))
      prediction = model.predict(x, verbose=0)
      print (int_to_char[np.argmax(prediction)], end = '')   #get next char index.
      seq_in = [int_to_char[value] for value in pattern]
      pattern.append(np.argmax(prediction))
      pattern = pattern[1:len(pattern)]

In [51]:
#picks a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print ("Seed -",  input_str, sep = '\n\n')
print ("\nGenerated string -\n")

predict_next_n_chars(pattern, 200)

Seed -

rrent right leave recurrent figure neural machine translation part input encoder consist two recurre

Generated string -

nt neural network also increase state compute gradient descent training data adaptation statistical machine translation system train example correct output word see figure illustration stack recurrent

In [52]:
input_str = "Many modern service systems rely on a network of hub facilities to help concentrate flows of freight or passengers"

#Uses the first 100 characters from given input_str as input to generate next 200 characters. 
input_str = input_str.lower()
input_string = ''
for each in input_str:
    if each in chars:
           if (len (input_string) < 100):
                input_string += each

pattern = []
pattern.append([char_to_int[char] for char in input_string])

print ("Seed -",  input_str, sep = '\n\n')
print ("\nGenerated string -\n")
predict_next_n_chars(pattern[0], 200)

Seed -

many modern service systems rely on a network of hub facilities to help concentrate flows of freight or passengers

Generated string -

er compute computationally concern implement use standard train example set train example self attention layer decoder state compute gradient descent training data adaptation statistical machine trans